In [2]:
# Core packages
import os
import time
from datetime import datetime, timedelta

# Data manipulation
import numpy as np
import pandas as pd

# Data sources
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Plot settings
%matplotlib inline
sns.set(rc={"figure.figsize": (12, 6)})

# ML / Stats (optional depending on modeling needs)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report


# Paths
RAW_DATA_DIR = "../data/raw/"
PROCESSED_DATA_DIR = "../data/processed/"
PLOTS_DIR = "../plots/"


In [14]:
#set Ticker
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TLT', 'XLE', 'EPD', 'BTC-USD']

#Master asset info dictionary
ticker_info = {}

for t in tickers:
    ticker_data = yf.Ticker(t)
    
    #dividend stock classification
    pays_div = not ticker_data.dividends.empty
    
    #classify asset type manually
    if t == 'BTC-USD':
        asset_type = 'crypto'
    elif t in ['TLT', 'XLE']:
        asset_type = 'etf'
    else:
        asset_type = 'equity'
        
    #Add to dictionary
    ticker_info[t] = {
        'type': asset_type,
        'pays_dividend': pays_div
    }
    
#preview results
import pprint
pprint.pprint(ticker_info)

#save info
import json

with open("../ticker_info.json", "w") as f:
    json.dump(ticker_info, f, indent = 4)

#load    
#with open("ticker_info.json", "r") as f:
#    ticker_info = json.load(f)


{'AAPL': {'pays_dividend': True, 'type': 'equity'},
 'BTC-USD': {'pays_dividend': False, 'type': 'crypto'},
 'EPD': {'pays_dividend': True, 'type': 'equity'},
 'GOOGL': {'pays_dividend': True, 'type': 'equity'},
 'MSFT': {'pays_dividend': True, 'type': 'equity'},
 'TLT': {'pays_dividend': True, 'type': 'etf'},
 'XLE': {'pays_dividend': True, 'type': 'etf'}}


In [4]:
# Set end and start dates as datetime objects
end_date = datetime.today()
start_date = end_date - timedelta(days=5*365)

#fetch data for each ticker
data = yf.download(tickers, start=start_date, end=end_date)

print(data.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  7 of 7 completed

Price            Close                                                  \
Ticker            AAPL       BTC-USD        EPD      GOOGL        MSFT   
Date                                                                     
2020-08-07         NaN  11601.472656        NaN        NaN         NaN   
2020-08-08         NaN  11754.045898        NaN        NaN         NaN   
2020-08-09         NaN  11675.739258        NaN        NaN         NaN   
2020-08-10  109.776489  11878.111328  12.533845  74.394829  199.521301   
2020-08-11  106.511749  11410.525391  12.451701  73.585693  194.855453   

Price                                    High                           ...  \
Ticker             TLT        XLE        AAPL       BTC-USD        EPD  ...   
Date                                                                    ...   
2020-08-07         NaN        NaN         NaN  11898.038086        NaN  ...   
2020-08-08         NaN        NaN         NaN  11800.064453        NaN  ...   
2020-08-09  

In [5]:
print(os.getcwd())

/mnt/c/Users/allana/OneDrive/Documents/Python Projects/Portfolio-Risk-Simulator/notebooks


In [6]:
#create directories for raw and processed data
os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Confirm path creation worked
print("Raw path exists:", os.path.exists(RAW_DATA_DIR))
print("Processed path exists:", os.path.exists(PROCESSED_DATA_DIR))

# Flatten MultiIndex columns 
data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]

#save data to csv
risk_portfolio = f"{RAW_DATA_DIR}multi_asset_5yr_data.csv"
data.to_csv(risk_portfolio)

print(f"Data saved to {risk_portfolio}")

Raw path exists: True
Processed path exists: True
Data saved to ../data/raw/multi_asset_5yr_data.csv


In [7]:
#reimport .csv to dataframe
csv_path = f"{RAW_DATA_DIR}multi_asset_5yr_data.csv"

#load data
df = pd.read_csv(csv_path, index_col=0, parse_dates=True)

#check values
print(df.shape)
print(df.columns)
df.head()

(1826, 35)
Index(['Close_AAPL', 'Close_BTC-USD', 'Close_EPD', 'Close_GOOGL', 'Close_MSFT',
       'Close_TLT', 'Close_XLE', 'High_AAPL', 'High_BTC-USD', 'High_EPD',
       'High_GOOGL', 'High_MSFT', 'High_TLT', 'High_XLE', 'Low_AAPL',
       'Low_BTC-USD', 'Low_EPD', 'Low_GOOGL', 'Low_MSFT', 'Low_TLT', 'Low_XLE',
       'Open_AAPL', 'Open_BTC-USD', 'Open_EPD', 'Open_GOOGL', 'Open_MSFT',
       'Open_TLT', 'Open_XLE', 'Volume_AAPL', 'Volume_BTC-USD', 'Volume_EPD',
       'Volume_GOOGL', 'Volume_MSFT', 'Volume_TLT', 'Volume_XLE'],
      dtype='object')


,Close_AAPL,Close_BTC-USD,Close_EPD,Close_GOOGL,Close_MSFT,Close_TLT,Close_XLE,High_AAPL,High_BTC-USD,High_EPD,...,Open_MSFT,Open_TLT,Open_XLE,Volume_AAPL,Volume_BTC-USD,Volume_EPD,Volume_GOOGL,Volume_MSFT,Volume_TLT,Volume_XLE
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-07,NaN,11601.472656,NaN,NaN,NaN,NaN,NaN,NaN,11898.038086,NaN,...,NaN,NaN,NaN,NaN,2.313231e+10,NaN,NaN,NaN,NaN,NaN
2020-08-08,NaN,11754.045898,NaN,NaN,NaN,NaN,NaN,NaN,11800.064453,NaN,...,NaN,NaN,NaN,NaN,1.757206e+10,NaN,NaN,NaN,NaN,NaN
2020-08-09,NaN,11675.739258,NaN,NaN,NaN,NaN,NaN,NaN,11806.056641,NaN,...,NaN,NaN,NaN,NaN,1.748961e+10,NaN,NaN,NaN,NaN,NaN
2020-08-10,109.776489,11878.111328,12.533845,74.394829,199.521301,146.511841,31.332096,110.796568,12045.140625,12.615990,...,202.797952,147.464797,30.620930,212403600.0,2.611411e+10,6863700.0,20546000.0,36716500.0,6408500.0,26308100.0
2020-08-11,106.511749,11410.525391,12.451701,73.585693,194.855453,144.770477,31.225826,109.537898,11932.710938,12.848732,...,198.477016,144.822454,31.945166,187902400.0,2.703978e+10,7558200.0,31098000.0,36446500.0,19234200.0,26513100.0


In [8]:
#check index 
print(type(df.index))
print(df.index[:5])  # show first few dates


<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10',
               '2020-08-11'],
              dtype='datetime64[ns]', name='Date', freq=None)


In [9]:
#check for missing values
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1826 entries, 2020-08-07 to 2025-08-06
Data columns (total 35 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Close_AAPL      1254 non-null   float64
 1   Close_BTC-USD   1824 non-null   float64
 2   Close_EPD       1254 non-null   float64
 3   Close_GOOGL     1254 non-null   float64
 4   Close_MSFT      1254 non-null   float64
 5   Close_TLT       1254 non-null   float64
 6   Close_XLE       1254 non-null   float64
 7   High_AAPL       1254 non-null   float64
 8   High_BTC-USD    1824 non-null   float64
 9   High_EPD        1254 non-null   float64
 10  High_GOOGL      1254 non-null   float64
 11  High_MSFT       1254 non-null   float64
 12  High_TLT        1254 non-null   float64
 13  High_XLE        1254 non-null   float64
 14  Low_AAPL        1254 non-null   float64
 15  Low_BTC-USD     1824 non-null   float64
 16  Low_EPD         1254 non-null   float64
 17  Low_GOOGL      

In [10]:
df.isnull().sum()

Close_AAPL        572
Close_BTC-USD       2
Close_EPD         572
Close_GOOGL       572
Close_MSFT        572
Close_TLT         572
Close_XLE         572
High_AAPL         572
High_BTC-USD        2
High_EPD          572
High_GOOGL        572
High_MSFT         572
High_TLT          572
High_XLE          572
Low_AAPL          572
Low_BTC-USD         2
Low_EPD           572
Low_GOOGL         572
Low_MSFT          572
Low_TLT           572
Low_XLE           572
Open_AAPL         572
Open_BTC-USD        2
Open_EPD          572
Open_GOOGL        572
Open_MSFT         572
Open_TLT          572
Open_XLE          572
Volume_AAPL       572
Volume_BTC-USD      2
Volume_EPD        572
Volume_GOOGL      572
Volume_MSFT       572
Volume_TLT        572
Volume_XLE        572
dtype: int64

In [11]:
# First, forward fill just to fill gaps from holidays/weekends
df = df.ffill()

# Then drop remaining NaNs per column + drop zero values per column
for col in df.columns:
    df = df[df[col].notna()]  # drop missing values in that column
    df = df[df[col] != 0]     # drop zero values in that column

print("Final shape:", df.shape)
print("Missing values left:\n", df.isnull().sum())

Final shape: (1823, 35)
Missing values left:
 Close_AAPL        0
Close_BTC-USD     0
Close_EPD         0
Close_GOOGL       0
Close_MSFT        0
Close_TLT         0
Close_XLE         0
High_AAPL         0
High_BTC-USD      0
High_EPD          0
High_GOOGL        0
High_MSFT         0
High_TLT          0
High_XLE          0
Low_AAPL          0
Low_BTC-USD       0
Low_EPD           0
Low_GOOGL         0
Low_MSFT          0
Low_TLT           0
Low_XLE           0
Open_AAPL         0
Open_BTC-USD      0
Open_EPD          0
Open_GOOGL        0
Open_MSFT         0
Open_TLT          0
Open_XLE          0
Volume_AAPL       0
Volume_BTC-USD    0
Volume_EPD        0
Volume_GOOGL      0
Volume_MSFT       0
Volume_TLT        0
Volume_XLE        0
dtype: int64


In [12]:
# Save cleaned data to processed folder
processed_path = os.path.join(PROCESSED_DATA_DIR, "multi_asset_5yr_data_cleaned.csv")
data.to_csv(processed_path)

print(f"Cleaned data saved to: {processed_path}")

Cleaned data saved to: ../data/processed/multi_asset_5yr_data_cleaned.csv
